In [17]:
# Using yahoo finance to get the stock data

import yfinance as yf

gold_data = yf.download("GC=F", start="2023-04-01", end="2023-07-09", interval="60m")

[*********************100%***********************]  1 of 1 completed


In [18]:
# Using oanda to get the data 
import requests
import json
import pandas as pd
import time 


API = "api-fxpractice.oanda.com"
STREAM_API = "stream-fxpractice.oanda.com"
ACCESS_TOKEN = "06a31b21abc8fb4cbe9bbd8c5b7b1f4e-260aefea008b878b6bffbfebafc84b9f"
ACCOUNT_ID = "101-004-26291813-001"


PRICING_PATH = f"/v3/accounts/{ACCOUNT_ID}/pricing"
query = {"instruments": "XAU_GBP"}
headers = {"Authorization": "Bearer " + ACCESS_TOKEN}
response = requests.get("https://" + API + PRICING_PATH, headers=headers, params=query)

In [19]:
# Creating a function to get the pricing information

def get_pricing(instrument):
    query = {"instruments": instrument}
    response = requests.get("https://" + API + PRICING_PATH, headers=headers, params=query)
    json_response = response.json()

    prices = json_response["prices"][0]["asks"][0]["price"]
    prices = float(prices)

    time_string = json_response["time"]
    time = pd.to_datetime(time_string)
    return time, prices

get_pricing("XAU_GBP")

(Timestamp('2023-07-11 11:45:50.874664837+0000', tz='UTC'), 1500.072)

In [20]:
# Get the historical data

# Time format is year, month, day
from_time = time.mktime(pd.to_datetime("2023-07-1-00-00-00").timetuple())
to_time = time.mktime(pd.to_datetime("2023-07-10-00-00-00").timetuple())

In [21]:
header = {"Authorization": "Bearer " + ACCESS_TOKEN}
# Granularity is limited for the amount of data available, no more than 3000 entries give or take
query = {"from": from_time, "to": to_time, "granularity": "M5"}

INSTRUMENT = "XAU_GBP"
CANDLES_PATH = f"/v3/accounts/{ACCOUNT_ID}/instruments/{INSTRUMENT}/candles"

response = requests.get("https://" + API + CANDLES_PATH, headers=header, params=query)

def json_to_pd(json):
    json_file = json.json()
    price_json = json_file["candles"]
    times = []
    close_price, high_price, low_price, open_price = [], [], [], []

    for candle in price_json:
        times.append(candle["time"])
        close_price.append(float(candle["mid"]["c"]))
        high_price.append(float(candle["mid"]["h"]))
        low_price.append(float(candle["mid"]["l"]))
        open_price.append(float(candle["mid"]["o"]))
    
    dataframe = pd.DataFrame({"close": close_price, "high": high_price, "low": low_price, "open": open_price})
    dataframe.index = pd.to_datetime(times)
    return dataframe

get_data = json_to_pd(response)